In [310]:
!pip install simpy plotly

In [311]:
import simpy

In [312]:
# 

In [313]:
class Job:
    def __init__(self, res, items=1):
        self.res = res
        self.items = items
        
    def __enter__(self):
        print("__enter__" )
        return self.res.get(self.items)

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__" )
        self.res.put(self.items)


In [314]:
class SystemResource(simpy.resources.container.Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.processing_data = []
        self.waiting_data = []

    def get(self, *args, **kwargs):
        #print(f"args {args}" )
        #print(f"args {kwargs}" )
        amount = args[0]
        print(f"received request resource - amount {amount} at %d" % self._env.now)
        print(f"level (available) {self.level} at %d" % self._env.now)
        print(f"{len(self.get_queue)} waiting at %d" % self._env.now)
        print(f"{self.used()} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.used()))
        self.waiting_data.append((self._env.now, len(self.get_queue)))
        return super().get(*args, **kwargs)

    def put(self, *args, **kwargs):
        amount = args[0]
        print(f"received release resource - amount {amount} at %d" % self._env.now)
        print(f"level (available) {self.level} at %d" % self._env.now)
        print(f"{len(self.get_queue)} waiting at %d" % self._env.now)
        print(f"{self.used()} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.used()))
        self.waiting_data.append((self._env.now, len(self.get_queue)))
        return super().put(*args, **kwargs)

    def used(self):
        return self.capacity - self.level

In [315]:
class User():
    def __init__(self, id, env, res):
        self.id = id
        self.env = env
        self.res = res
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = env.process(self.run())

    def run(self):
        while True:
            taskname = "process_task"
            print(f"starts task at %d" % self.env.now)
            #task_duration = 5
            task_duration = 2
            yield self.env.process(self.process_task(task_duration))
            print(f"ends task at %d" % self.env.now)

    def process_task(self, duration):
        with Job(self.res,2) as req:
            yield req
            yield self.env.timeout(duration)


In [316]:
env = simpy.Environment()

res = SystemResource(env, init=10, capacity=10)
user = User(1, env, res)
env.run(until=5)

print("Processing")
print(res.processing_data)
print("Waiting")
print(res.waiting_data)

starts task at 0
__enter__
received request resource - amount 2 at 0
level (available) 10 at 0
0 waiting at 0
0 processing at 0
__exit__
received release resource - amount 2 at 2
level (available) 8 at 2
0 waiting at 2
2 processing at 2
ends task at 2
starts task at 2
__enter__
received request resource - amount 2 at 2
level (available) 10 at 2
0 waiting at 2
0 processing at 2
__exit__
received release resource - amount 2 at 4
level (available) 8 at 4
0 waiting at 4
2 processing at 4
ends task at 4
starts task at 4
__enter__
received request resource - amount 2 at 4
level (available) 10 at 4
0 waiting at 4
0 processing at 4
Processing
[(0, 0), (2, 2), (2, 0), (4, 2), (4, 0)]
Waiting
[(0, 0), (2, 0), (2, 0), (4, 0), (4, 0)]


In [317]:
def user_generator(size, env, res):
    for i in range(size):
        User(i, env, res).action
        yield env.timeout(5)

In [319]:
# au passage suppression de magix numbers
sim_duration = 10
nb_users = 2
capacity = 2

env = simpy.Environment()
res = SystemResource(env, init=capacity, capacity=capacity)
#user = User(env, res)
user_gen = env.process(user_generator(nb_users, env, res))

env.run(until=sim_duration)



starts task at 0
__enter__
received request resource - amount 2 at 0
level (available) 2 at 0
0 waiting at 0
0 processing at 0
__exit__
received release resource - amount 2 at 2
level (available) 0 at 2
0 waiting at 2
2 processing at 2
ends task at 2
starts task at 2
__enter__
received request resource - amount 2 at 2
level (available) 2 at 2
0 waiting at 2
0 processing at 2
__exit__
received release resource - amount 2 at 4
level (available) 0 at 4
0 waiting at 4
2 processing at 4
ends task at 4
starts task at 4
__enter__
received request resource - amount 2 at 4
level (available) 2 at 4
0 waiting at 4
0 processing at 4
starts task at 5
__enter__
received request resource - amount 2 at 5
level (available) 0 at 5
0 waiting at 5
2 processing at 5
__exit__
received release resource - amount 2 at 6
level (available) 0 at 6
1 waiting at 6
2 processing at 6
ends task at 6
starts task at 6
__enter__
received request resource - amount 2 at 6
level (available) 0 at 6
0 waiting at 6
2 processin

In [ ]:
print("Processing")
print(res.processing_data)


In [ ]:
print("Waiting")
print(res.waiting_data)